## FFT for Image Classification

### Image transformation - Spatial to Frequency Domain

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

image_path='cats.jpg'

# Loading the image from given path
img=Image.open(image_path)
# Converting the image into gray-scale
img_gray=np.array(img.convert('L'))

# Computing FFT

# Applying 2D FFT to gray scale image to convert it into frequency domain
fft_image=np.fft.fft2(img_gray)
# Shifting the zero frequency component (DC) to the center
# FFT puts low frequencies at corners, so shift makes visualization clear
fft_image_shifted=np.fft.fftshift(fft_image)

# Computing magnitude spectrum

# np.abs(fshift): get the magnitude of the complex numbers (ignore phase)
# np.log(...): apply logarithm to reduce the wide range of values for better visualization
# +1: to avoid log(0), which would give negative infinity
# Multiply by 20: a scaling factor (common in frequency spectrum visualization)
magnitude_spectrum=20*np.log(np.abs(fft_image_shifted)+1)

# Displaying original image and magnitude spectrum
plt.figure(figsize=(12, 6))

plt.subplot(121), plt.imshow(img_gray, cmap='gray'), plt.title("Original Image")
plt.subplot(122), plt.imshow(magnitude_spectrum, cmap='gray'), plt.title("Magnitude Spectrum")
plt.show()

### Low Pass Filter

In [ ]:
# Creating a mask with a low-pass filter
rows, cols=img_gray.shape
c_row, c_col=rows//2, cols//2
mask=np.zeros((rows, cols), np.uint8)
mask[c_row-30:c_row+30, c_col-30:c_col+30]=1

f_shift_masked=fft_image_shifted*mask

# Move zero-frequency back to the original position
f_ishift=np.fft.ifftshift(f_shift_masked)

# Apply inverse FFT to get back spatial domain
image_filtered=np.fft.ifft2(f_ishift)
# Take only magnitude to discard imaginary part
image_filtered=np.abs(image_filtered)

plt.imshow(image_filtered, cmap='gray')
plt.title("Low-Pass Image")
plt.show()

### High Pass Filter

In [ ]:
rows, cols=img_gray.shape
c_row, c_col=rows//2, cols//2
mask=np.ones((rows, cols), np.uint8)
mask[c_row-30:c_row+30, c_col-30:c_col+30]=0

fft_shifted_masked=fft_image_shifted*mask
f_ishift=np.fft.ifftshift(fft_shifted_masked)

image_filtered=np.fft.ifft2(f_ishift)
image_filtered=np.abs(image_filtered)

plt.imshow(image_filtered, cmap='gray')
plt.title("High-Pass Image")
plt.show()

### Image Compression

In [ ]:
# Zero out small frequencies to compress the image

# Low threshold - keep more details (less compression)
# High threshold - keep only strong structures (more compression)

# Take threshold as 10% of maximum frequency magnitude
threshold=0.1*np.max(np.abs(fft_image_shifted))
fshift_compressed=np.where(np.abs(fft_image_shifted)>threshold, fft_image_shifted, 0)

f_ishift=np.fft.ifftshift(fshift_compressed)
compressed_image=np.fft.ifft2(f_ishift)
compressed_image=np.abs(compressed_image)

plt.imshow(compressed_image, cmap='gray')
plt.title("Compressed Image")
plt.show()

### Edge Detection

In [ ]:
# Laplacian filter for edge detection in frequency domain

rows,cols=img_gray.shape
# These will be useful for padding the Laplacian filter so it aligns with the image size
c_row, c_col=rows//2, cols//2

# Spatial Domain Laplacian Kernel (3x3)
# Emphasizes regions of rapid intensity change (edges) by computing the second derivative of the image
laplacian=np.array([[0, 1, 0],
                    [1, -4, 1],
                    [0, 1, 0]])
# Pad the Laplacian kernel to match the size of the image
laplacian=np.pad(laplacian,
                 [(c_row-1, rows-c_row-2), (c_col-1, cols-c_col-2)],
                 mode='constant')

fft_laplacian=np.fft.fft2(laplacian)
# Convolution in spatial domain = Multiplication in frequency domain
filtered_img=fft_image*fft_laplacian

image_edge=np.fft.ifft2(filtered_img)
image_edge=np.abs(image_edge)

plt.imshow(image_edge, cmap='gray')
plt.title("Edge Detected Image")
plt.show()

### Pattern Detection

In [ ]:
import requests
from io import BytesIO

template_url="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT-xwX0-p5UlqDFP1hOnU9uGU5yV0hbXhZEmL_jpJINqgxpOw4uzh5Wg6Y5&s"

response_template=requests.get(template_url)
template_data=response_template.content
template_pil=Image.open(BytesIO(template_data)).convert('L')

template=np.array(template_pil)
template_fft=np.fft.fft2(template, s=img_gray.shape)

# Cross-correlation in frequency domain
# Correlation in spatial domain = Multilication with conjugate in frequency domain
correlation=np.fft.ifft2(fft_image*np.conj(template_fft))
correlation=np.abs(correlation)

threshold=0.8*np.max(correlation)

# Check if the template is significantly present
if np.any(correlation>threshold):
    # If any pixel in correlation map exceeds threshold → strong match found
    print("Pattern detected")
else:
    print("Pattern not detected")

### Blur Detection

In [ ]:
magnitude_spectrum=np.log(np.abs(fft_image_shifted)+1)

# Measure high frequency energy
# np.percentile(..., 99): finds intensity cutoff at the top 1% of frequency values (very high frequencies)
# magnitude_spectrum > that cutoff: creates a mask of "strong high-frequency components"
# np.sum(...): counts how many high-frequency values exist (measures edge/detail content)

high_freq_energy=np.sum(magnitude_spectrum>np.percentile(magnitude_spectrum, 99))

threshold=1000
# If high-frequency energy is below this → image lacks details → blurred

if high_freq_energy<threshold:
    print("Image is blurred")
else:
    print("Image is sharp")